## GRIB: using array fieldlists

In this example we will use a GRIB file containing 4 messages. First we ensure the file is available and read it into a fieldlist.

In [1]:
import earthkit.data as ekd
ekd.download_example_file("test4.grib")
ds_in = ekd.from_source("file", "test4.grib")

### Numpy array fieldlist

The "numpy" fieldlist we generate in the cell below works exactly in the same way as the original one but stores all the data in memory.

In [2]:
ds = ds_in.to_fieldlist()

In [3]:
len(ds)

4

### Pytorch array fieldlist

For the next example we choose the "pytorch" array backend. Since pytorch is an optional dependency for earthkit-data we need to ensure it is installed in the environment.

In [4]:
!pip install torch --quiet

In [5]:
ds = ds_in.to_fieldlist(array_backend="pytorch")

In [6]:
ds.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
1,ecmf,z,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
2,ecmf,t,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll
3,ecmf,z,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll


#### values

In [7]:
ds[0].values[:10]

tensor([228.0460, 228.0460, 228.0460, 228.0460, 228.0460, 228.0460, 228.0460,
        228.0460, 228.0460, 228.0460], dtype=torch.float64)

In [8]:
ds[0].values.shape

torch.Size([65160])

In [9]:
ds.values.shape

torch.Size([4, 65160])

#### to_array()

In [10]:
ds[0].to_array()[:2,:2]

tensor([[228.0460, 228.0460],
        [228.6085, 228.5792]], dtype=torch.float64)

In [11]:
ds.to_array().shape

torch.Size([4, 181, 360])

In [12]:
ds.to_array(flatten=True).shape

torch.Size([4, 65160])

#### to_numpy()

In [13]:
ds[0].to_numpy()[:2,:2]

array([[228.04600525, 228.04600525],
       [228.60850525, 228.57920837]])

In [14]:
ds.to_numpy().shape

(4, 181, 360)

### Building array fieldlists with from_array()

In [15]:
md = ds.metadata()
v = ds.to_array() + 2
r1 = ekd.FieldList.from_array(v, md)
r1.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
1,ecmf,z,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
2,ecmf,t,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll
3,ecmf,z,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll


As expected, the values in *r1* are now differing by 2 from the ones in the original fieldlist (*r*).

In [16]:
r1[0].values[:10]

tensor([230.0460, 230.0460, 230.0460, 230.0460, 230.0460, 230.0460, 230.0460,
        230.0460, 230.0460, 230.0460], dtype=torch.float64)

### Building an array fieldlist in a loop

In [17]:
md = ds.metadata()
v = ds.to_array() + 2

r1 = ekd.SimpleFieldList()
for k in range(len(md)):
    r1.append(ekd.ArrayField(v[k], md[k]))
r1.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
1,ecmf,z,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
2,ecmf,t,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll
3,ecmf,z,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll


### Saving to GRIB

In [18]:
path = "_from_pytroch.grib"
r1.to_target("file", path)
ds1 = ekd.from_source("file", path)
ds1.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
1,ecmf,z,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll
2,ecmf,t,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll
3,ecmf,z,isobaricInhPa,850,20070101,1200,0,an,0,regular_ll
